In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
# Import data
segmentData = pd.read_csv("./data/Sam/all_segments.csv")

# Remove irrelevent columns
segmentColumnsToRemove = ['confidence', 'loudness_end', 'track_title', 'album_title', 'album_artist', 'order', 'pitches']
segmentData.drop(columns = segmentColumnsToRemove, axis = 1, inplace = True)

# Convert Track Title to unique number
segmentTrackIds = segmentData['track_ids'].astype('category') # Extract the song title column
segmentDataCodes = segmentTrackIds.cat.codes # Assign each song title to a unique number
segmentData['track_title'] = segmentDataCodes # Replace track title with unique number

# Initialise output matrix
allSongEuclidianTimbreHeadings = ['song_timbre', 'song_timbre_start', 'song_timbre_end', 'loudness_start', 'loudness_end']
allSongEuclidianTimbre = pd.DataFrame(np.zeros((max(segmentDataCodes), 5)), columns = allSongEuclidianTimbreHeadings) 

i = 0
for song in range(0, max(segmentDataCodes)+1):

    if i%100 == 0:
        print(str(i/4949 * 100) + ' percent complete')

    songSegmentData = segmentData.loc[segmentData['track_title'] == song] # Extract segments for one song

    # Find number of segments in first 10 secs of song
    time = 0
    numStartSegments = 0
    while time < 10:
        time += songSegmentData.iloc[numStartSegments,1]
        numStartSegments += 1

    # Find number of segments in last 10 secs of song
    time = 0
    numEndSegments = 0
    while time < 10:
        time += songSegmentData.iloc[-numStartSegments,1]
        numEndSegments += 1

    # Timbre
    timbreStr = songSegmentData['timbres'] # Extract Timbre column vector from segmentData
    timbre = pd.DataFrame(np.zeros((12, len(timbreStr)))) # Initialise timbre matrix
    # Separate out each value of the Timbre column vector, and save to a column of the timbre matrix (then transpose)
    x=0
    for row in timbreStr:
        timbre[x] = pd.DataFrame(row.split(', '))
        x+=1
    timbre = timbre.transpose()
    # Remove the square brackets from start and end of the data rows
    timbre[0] = timbre[0].str[1:]
    timbre[11] = timbre[11].str[:-1]

    timbre = timbre.astype('float64') # Convert timbre matrix 

    # Weight timbre dimensions in order accourding to importance
    weightedTimbre = pd.DataFrame(np.zeros((len(timbre), 12))) 
    for x in range(1,12):
        weightedTimbre[x] = timbre[x]*(0.9**x)

    # For each row of timbre values find the euclidean distance of the 12 variables
    euclidianTimbre = pd.DataFrame(np.zeros((1, len(timbre))))
    for x in range(len(timbre)):
        euclidianTimbre[x] = np.linalg.norm(weightedTimbre.loc[x,:])
    euclidianTimbre = euclidianTimbre.transpose()

    # Normalise the data between zero and one
    euclidianTimbre = pd.DataFrame(MinMaxScaler(feature_range=(0,1)).fit_transform(euclidianTimbre))
    # euclidianTimbre.plot.line()

    # Average for euclidian distance for song
    euclidianTimbreSong = float(np.mean(euclidianTimbre, axis=0))
    # Average for the first 10 secs of the song
    euclidianTimbreSongStart = float(np.mean(euclidianTimbre.head(numStartSegments), axis=0))
    # Average for the last 10 secs of the song
    euclidianTimbreSongEnd = float(np.mean(euclidianTimbre.tail(numEndSegments), axis=0))

    # Loudness
    songLoudness = songSegmentData['loudness_max']
    # Average for the first 10 secs of the song
    loudnessSongStart = float(np.mean(songLoudness.head(numStartSegments), axis=0))
    # Average for the last 10 secs of the song
    loudnessSongEnd = float(np.mean(songLoudness.tail(numEndSegments), axis=0))

    allSongEuclidianTimbre.loc[song,:] = [euclidianTimbreSong, euclidianTimbreSongStart, euclidianTimbreSongEnd, loudnessSongStart, loudnessSongEnd] # Export values

    i += 1

allSongEuclidianTimbre

0.0 percent complete
2.020610224287735 percent complete
4.04122044857547 percent complete
6.061830672863205 percent complete
8.08244089715094 percent complete
10.103051121438675 percent complete
12.12366134572641 percent complete
14.144271570014144 percent complete
16.16488179430188 percent complete
18.185492018589613 percent complete
20.20610224287735 percent complete
22.226712467165083 percent complete
24.24732269145282 percent complete
26.267932915740555 percent complete
28.28854314002829 percent complete
30.30915336431602 percent complete
32.32976358860376 percent complete
34.35037381289149 percent complete
36.37098403717923 percent complete
38.39159426146696 percent complete
40.4122044857547 percent complete
42.432814710042436 percent complete
44.453424934330165 percent complete
46.4740351586179 percent complete
48.49464538290564 percent complete
50.515255607193374 percent complete
52.53586583148111 percent complete
54.55647605576884 percent complete
56.57708628005658 percent comp

,song_timbre,song_timbre_start,song_timbre_end,loudness_start,loudness_end
0,0.410775,0.502790,0.381447,-50.058500,-50.912
1,0.288576,0.358865,0.420081,-40.046167,-46.607
2,0.295317,0.452228,0.415978,-14.745556,-42.064
3,0.292235,0.431800,0.463382,-38.945889,-58.058
4,0.195611,0.382496,0.894774,-14.908800,-0.241
...,...,...,...,...,...
4944,0.140222,0.303244,0.675694,-8.244444,-16.261
4945,0.156536,0.221387,0.418692,-48.704600,-48.996
4946,0.272332,0.456071,0.726044,-21.911778,-56.151
4947,0.172822,0.095022,1.000000,-16.969600,-5.312


In [4]:
allSongEuclidianTimbre.to_csv('./data/timbre.csv')

In [5]:
data = pd.read_csv('./data/data.csv')
data = pd.concat([data, allSongEuclidianTimbre], axis=1)

data.to_csv('./data/data_timbre.csv')

In [6]:
# len(segmentData['track_ids'].unique())
data = pd.read_csv('./data/data.csv')

prev_ids = []

for id in data['id']:
    prev_ids.append(id)
    if len(np.unique(prev_ids)) != len(prev_ids):
        print(id)
        1/0

print(len(np.unique(prev_ids)))
print(len(prev_ids))


4949
4949
